<a href="https://colab.research.google.com/github/squadOito/soulcodead2/blob/joseaureliok%2Fnotebook/notebooks/notebook_mongoDB_carregamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Alunos: Nomes, Nomes**

Professores: Douglas Ribeiro e Franciane Rodrigues

## Preparação de Ambiente

In [1]:
# Instalação bibliotecas
!pip install -q pymongo
!pip install -q gcsfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 7.8 MB/s eta 0:00:00


In [2]:
# Importação de bibliotecas
import os
import pandas as pd
import numpy as np

from google.colab import data_table
from google.cloud import storage
from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pymongo import MongoClient

### Função Chave

In [3]:
def connectKey (chave):
  '''
  Conecta ao drive para obtenção de chave
  chave = nome da chave em string
  '''
  # Cria compartilhamento com Google Drive
  drive.mount('/content/drive', force_remount=True)

  # Arquivo a ser acessado na pasta compartilhada
  target = chave

  # Caminho completo da pasta compartilhada
  folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

  # Acesso ao arquivo no colab
  return os.path.join(folder, target)



### Compartilhamento Chave GDrive

In [4]:
# Nome da chave a ser acessada
chaveGcp = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Chave GCP
serviceAccount = connectKey(chaveGcp)

Mounted at /content/drive


In [5]:
# Nome da chave a ser acessada
chaveMongo = 'X509-cert-5608832275999674572.pem'

# Chave MongoDB
mongoKey = connectKey(chaveMongo)

Mounted at /content/drive


### Configuração Conexão MongoDB

In [6]:
# configurando conexão
uri = "mongodb+srv://energy.drstakp.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,
					 tls=True,
					 tlsCertificateKeyFile= mongoKey)

### Configuração da bucket

In [7]:
# Nome da bucket
bucket_name = 'projeto-final-ad2-e8'

# Caminho do prefixo
prefix_name = 'dados/tratados/'

In [8]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [9]:
# Conexão com a bucket do Google Cloud
clientGcp = storage.Client()
bucket = clientGcp.get_bucket(bucket_name)

### Conjuntos de dados

In [10]:
# Formatação tabular
data_table.enable_dataframe_formatter()

#data_table.disable_dataframe_formatter()

In [11]:
# Listando conjuntos de dados na bucket
blobList = [x.name for x in bucket.list_blobs(prefix = prefix_name)]

# dataframe lista blobs
blobDf = pd.DataFrame(blobList)

blobDf

,0
0,dados/tratados/
1,dados/tratados/Veiculos_eletricos_brasil_anfav...
2,dados/tratados/dataset_tratados_xlsx/
3,dados/tratados/dataset_tratados_xlsx/2_1_cap_i...
4,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
5,dados/tratados/dataset_tratados_xlsx/2_2_cap_i...
6,dados/tratados/dataset_tratados_xlsx/2_3_gerac...
7,dados/tratados/fator_capacidade_mwh_2015_2023.csv
8,dados/tratados/iea_ponto_recarga_tratado.csv
9,dados/tratados/iea_veiculos_tratado.csv


### Seleção do conjunto de dados

In [12]:
# Seleção do blob - CSV
blobC1 = 'dados/tratados/fator_capacidade_mwh_2015_2023.csv'
blobC2 = 'dados/tratados/tratado_geracao_distribuida.csv'
blobC3 = 'dados/tratados/iea_ponto_recarga_tratado.csv'
blobC4 = 'dados/tratados/Veiculos_eletricos_brasil_anfavea_2012_2023_crescimento.csv'
blobC5 = 'dados/tratados/tratado_global_horizontal_means.csv'
blobC6 = 'dados/tratados/dataset_tratados_xlsx/2_2_cap_instalada_de_geracao_eletrica_por_fonte_mw.csv'

# Seleção do blob - EXCEL
blobE1 = 'dados/tratados/dataset_tratados_xlsx/2_1_cap_instalada_por_regiao_e_uf_tratado.xlsx'
blobE3 = 	'dados/tratados/dataset_tratados_xlsx/2_3_geracao_eletrica_por_fonte_gwh_tratado.xlsx'

In [13]:
# Caminho gsutil do arquivo

pathC = {
        'fator_capacidade' : f'gs://{bucket_name}/{blobC1}',
        'geracao_distribuida' : f'gs://{bucket_name}/{blobC2}',
        'iea_recarga' : f'gs://{bucket_name}/{blobC3}',
        'veiculos_anfeava' : f'gs://{bucket_name}/{blobC4}',
        'global_means' : f'gs://{bucket_name}/{blobC5}',
        'instalada_fonte' : f'gs://{bucket_name}/{blobC6}'
}

pathE = {
        'instalada_regiao' : f'gs://{bucket_name}/{blobE1}',
        'geracao_fonte' : f'gs://{bucket_name}/{blobE3}'
}

## Collection

In [14]:
# Criando instância client
db = client['backup']

# Criando coleções
collection_csv = db['csv']
collection_excel = db['excel']

### Verificação

In [15]:
# Verificando valores no MongoDB
doc_count_csv = collection_csv.count_documents({})
doc_count_excel = collection_excel.count_documents({})

print( f'\n-CSV: {doc_count_csv} \n-EXCEL: {doc_count_excel}')


-CSV: 0 
-EXCEL: 0


## Integração Google Cloud Storage - MongoDB

### Extração da bucket

#### CSV

In [16]:
dictC = {}

for k, v in pathC.items():
  dfC = pd.read_csv(v)

  dictC[k] = dfC

print(dictC)

<ipython-input-16-c2864569bb55>:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  dfC = pd.read_csv(v)


{'fator_capacidade':         id_subsistema        nom_subsistema id_estado nom_estado  \
0                  NE              Nordeste        BA      BAHIA   
1                  NE              Nordeste        BA      BAHIA   
2                  NE              Nordeste        BA      BAHIA   
3                  NE              Nordeste        BA      BAHIA   
4                  NE              Nordeste        BA      BAHIA   
...               ...                   ...       ...        ...   
8622775            SE  Sudeste/Centro-Oeste        SP  SAO PAULO   
8622776            SE  Sudeste/Centro-Oeste        SP  SAO PAULO   
8622777            SE  Sudeste/Centro-Oeste        SP  SAO PAULO   
8622778            SE  Sudeste/Centro-Oeste        SP  SAO PAULO   
8622779            SE  Sudeste/Centro-Oeste        SP  SAO PAULO   

                   nom_pontoconexao nom_localizacao  val_latitudesecoletora  \
0            IGAPORA II - 69 kV (A)        Interior              -14.208409   
1   

#### EXCEL

In [23]:
dictE = {}

for k, v in pathE.items():
  dfE = pd.read_csv(v)

  dictE[k] = dfE

print(dictE)

{'instalada_regiao':               Grupo Macro_grupo         Ano   Tipo_de_informacao  \
0            Brasil      Brasil  2013-01-01  Cap. Instalada (MW)   
1            Brasil      Brasil  2013-01-01        Geração (GWh)   
2            Brasil      Brasil  2014-01-01  Cap. Instalada (MW)   
3            Brasil      Brasil  2014-01-01        Geração (GWh)   
4            Brasil      Brasil  2015-01-01  Cap. Instalada (MW)   
..              ...         ...         ...                  ...   
555  Santa Catarina         Sul  2020-01-01        Geração (GWh)   
556  Santa Catarina         Sul  2021-01-01  Cap. Instalada (MW)   
557  Santa Catarina         Sul  2021-01-01        Geração (GWh)   
558  Santa Catarina         Sul  2022-01-01  Cap. Instalada (MW)   
559  Santa Catarina         Sul  2022-01-01        Geração (GWh)   

             Total  
0    126743.303060  
1    570834.647328  
2    133912.521110  
3    590542.338656  
4    140857.748378  
..             ...  
555   16590.335

### Carregamento na coleção

In [25]:
# geracao_fonte
collection_excel.insert_many(dictE['geracao_fonte'].to_dict("records"))

print('Done')

Done


In [26]:
# instalada_regiao
collection_excel.insert_many(dictE['instalada_regiao'].to_dict("records"))

print('Done')

Done


In [27]:
# instalada_fonte
collection_csv.insert_many(dictC['instalada_fonte'].to_dict("records"))

print('Done')

Done


In [29]:
# veiculos_anfavea
collection_csv.insert_many(dictC['veiculos_anfeava'].to_dict("records"))

print('Done')

Done


In [30]:
# global_means
collection_csv.insert_many(dictC['global_means'].to_dict("records"))

print('Done')

Done


In [32]:
# iea_recarga
collection_csv.insert_many(dictC['iea_recarga'].to_dict("records"))

print('Done')

Done


Os últimos dois conjuntos de dados, a seguir, em razão do volume de dados e do armazenamento disponível no MongoDB para contas gratuitas, foram reduzidos, extraindo-se amostragem de 10 mil linhas para cada, para fins de disponibilização de backup

In [36]:
# geracao_distribuida
collection_csv.insert_many((dictC['geracao_distribuida'].sample(10000)).to_dict("records"))

print('Done')

Done


In [37]:
# fator_capacidade
collection_csv.insert_many((dictC['fator_capacidade'].sample(10000)).to_dict("records"))

print('Done')

Done


#### Verificação

In [38]:
# Verificando valores no MongoDB
doc_count_csv = collection_csv.count_documents({})
doc_count_excel = collection_excel.count_documents({})

print( f'\n-CSV: {doc_count_csv} \n-EXCEL: {doc_count_excel}')


-CSV: 93014 
-EXCEL: 660


# Status

In [39]:
print("BACKUP CONCLUÍDO COM SUCESSO!!!")

BACKUP CONCLUÍDO COM SUCESSO!!!
